### Connect to Google colab 
<a href="https://colab.research.google.com/github/ithabibi/Persian-Opinion-Mining-and-Sentiment-Analysis/blob/main/query-Sentiment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Persian Sentiment Analysis With Fasttext language Model and LSTM convolutional neural network(CNN)
### Persian sentiment analysis step by step guide


---


#### so there are 5 steps we going through with each other 

## Step 1) Choose and Preparing Word Embedding Model

in this step we gonna to prepare word embedding model.(https://towardsdatascience.com/introduction-to-word-embedding-and-word2vec-652d0c2060fa) 
there are too many ways to train a word embedding model for example :

1.   Fasttext
2.   ELMo (Embeddings from Language Models)
3.   Universal Sentence Encoder 
4.   Word2Vec
5.   GloVe (Global Vector)

if you Want to know more then read [this article from Thomas Wolf](https://medium.com/huggingface/universal-word-sentence-embeddings-ce48ddc8fc3a) but now we gonna use Fasttext because it's Pretrained by Facebook and we can use it ( there is nothing to worry about this model it's pretty easy to train it by your self or your corpus facebook used Persian Wikipedia and some other staff as dataset for this model so it's just very simpler for us)

In [ ]:
#@title Python Package and library version

pip install pybind11==2.11.1
#pip install fasttext==0.9.2 #For Word Embedding Model 
pip install fasttext-wheel # Alternative For pip install fasttext

pip install tensorflow==2.12.0 #For Deep Learning
pip install keras==2.12.0 #A wrapper for TensorFlow for simplicity

pip install pandas==1.5.3
pip install numpy==1.23 
pip install hazm==0.7.0 #For NLP processing



In [1]:
import pandas
import random
import numpy
import hazm 
import fasttext

d:\IT PHD 1403\Sentiment-Analyzer\Persian-Opinion-Mining-and-Sentiment-Analysis\myenv\lib\site-packages\nltk\metrics\scores.py:18: UserWarning: A NumPy version >=1.23.5 and <2.5.0 is required for this version of SciPy (detected version 1.23.0)
  from scipy.stats.stats import betai


In [2]:
%time
fasttext_model = fasttext.load_model("cc.fa.300.bin")

CPU times: total: 0 ns
Wall time: 0 ns


In [ ]:
#@title Unit test Fasttext word embedding model by similar word

phrase = "\u062A\u062A\u0648" #@param {type:"string"}
print("dimension of " + phrase + " is " +str(fasttext_model.get_dimension()))
print(fasttext_model.get_word_vector(phrase).shape)
print(fasttext_model[phrase]) # get the vector of the word 

# get similar word
fasttext_model.get_nearest_neighbors(phrase)

## Step 2) Dataset Normalization and Preparation

in this step we going to collect a dataset that crawled by [@minasmz](https://github.com/minasmz) it's not good and I only used 450 pos and 450 neg reviews from it.anyway here we will download the dataset and split it to train and test ( I created Train and Test then I filled it with data )

### Define train_size and test_size

In [ ]:
#@title Upload on google colab and prepare Dataset
#!rm -rf /content/related-query-whit-lexion.csv
#!wget https://raw.githubusercontent.com/ithabibi/Persian-Opinion-Mining-and-Sentiment-Analysis/main/related-query-whit-lexion.csv

# load and read sentiment_tagged dataset.csv file in tne path ./content/ in google colab. 
# this dataset include three element: Comment,Score,Suggestion. Comment is feature and Suggestion is label.
import time
import winsound

# باز کردن فایل با بافرینگ خطی (هر خط بلافاصله نوشته شود)
with open("output.txt", "w", buffering=1) as file:
    for i in range(10, 301, 10):

      csv_dataset = pandas.read_csv("related-query-whitout-lexion.csv")

      def CleanPersianText(text):
        _normalizer = hazm.Normalizer()
        text = _normalizer.normalize(text)
        return text

      # Cleansing the dataset and creating a new list with two elements: "Comment" and "suggestion"filde. (but without the third element: "score")
      # The new list is created by the zip command --> x= zip(csv_dataset['Comment'],csv_dataset['Suggestion'])
      # valu of suggestion is 1,2,3 or positive,negative,neutral
      revlist = list(map(lambda x: [CleanPersianText(x[0]),x[1]],zip(csv_dataset['Query'],csv_dataset['Suggestion'])))

      # Separation of positive and negative suggestions
      positive=list(filter(lambda x: x[1] == 1,revlist))
      neutral=list(filter(lambda x: x[1] == 3,revlist))
      negative=list(filter(lambda x: x[1] == 2,revlist))

      # print number of element exist in positive, neutral, negative, revlist list 
      print("*" * 88)
      print("Posetive count {}".format(len(positive)))
      print("*Negetive count {}".format(len(negative)))
      print("Natural  count {}".format(len(neutral)))
      print("Total dataset count {}".format(len(revlist)))

      # mix positive and negative suggestions for 1438 element.
      # We chose 438 because the most negative comments were 438 = "len(negative)"
      revlist_shuffle = positive[:1000] + negative[:304]
      random.shuffle(revlist_shuffle)
      random.shuffle(revlist_shuffle)#double shuffle
      print("Total shuffle count {}".format(len(revlist_shuffle)),"\n")

      # print random element from positive, neutral, negative List
      print("Random Posetive Query: ","\n",positive[random.randrange(1,len(positive))])
      print("Random Negetive Query: ","\n",negative[random.randrange(1,len(negative))])
      print("Random unknown  Query: ","\n",neutral[random.randrange(1,len(neutral))])
      #--------------------------------------------
      #@title create and Prepare Train & Test data_structure with zero value
      embedding_dim = 300 #@param {type:"integer"}
      max_vocab_token = 8 #@param {type:"integer"} #set 5 for related query in google trends
      import numpy as np
      import keras.backend as K

      train_size = int(0.90*(len(revlist_shuffle)))
      test_size = int(0.10*(len(revlist_shuffle)))

      # x_train same as features and y_train same as the label. x_train same as input and y_train same as output.
      # The x_train data have 3 Dimention (874,10,300): (number_of_comment,number_of_words, dimension_of_fasttext)
      # The y_train data has 2 dimensions (874,2): (number of comments, suggestions)
      # The suggestions are 1 or 3. 1's are positive and 3's are negative suggestions.
      x_train = np.zeros((train_size, max_vocab_token, embedding_dim), dtype=K.floatx())
      y_train = np.zeros((train_size, 2), dtype=np.int32)

      x_test = np.zeros((test_size, max_vocab_token, embedding_dim), dtype=K.floatx())
      y_test = np.zeros((test_size, 2), dtype=np.int32)

      #-----------------------------------------------
      #@title Fill X_Train, X_Test, Y_Train, Y_Test by Dataset
      indexes = set(np.random.choice(len(revlist_shuffle), train_size + test_size, replace=False)) # for random selection
      print("data_item is: " + str(len(indexes)),"\n")

      for data_item, index in enumerate(indexes): # indexes include 920 items of comments
        comment = hazm.word_tokenize(revlist_shuffle[index][0]) #[0] means the "comment" field in the .csv file
        for vocabs in range(0,len(comment)):
          if vocabs >= max_vocab_token: 
            break # If the comment is more than twenty words, only the first twenty words will be considered
          if comment[vocabs] not in fasttext_model.words:
            continue # If vocab does not exist in fasttext, every 300 elements of that word's vector in x_train is zero
          if data_item < train_size:
            x_train[data_item, vocabs, :] = fasttext_model.get_word_vector(comment[vocabs])
          else:
            x_test[data_item - train_size, vocabs, :] = fasttext_model.get_word_vector(comment[vocabs])

        if data_item < train_size:
          y_train[data_item, :] = [1.0, 0.0] if revlist_shuffle[index][1] == 2 else [0.0, 1.0]
        else:
          y_test[data_item - train_size, :] = [1.0, 0.0] if revlist_shuffle[index][1] == 2 else [0.0, 1.0]
          
      print (x_train.shape,x_test.shape,y_train.shape,y_test.shape)

      #-------------------------------------------
      #@title Building Layers of LSTM Model
      from keras.models import Sequential
      from keras.layers import Conv1D, Dropout, Dense, Flatten, LSTM, MaxPooling1D, Bidirectional
      from keras.optimizers import Adam
      from keras.callbacks import EarlyStopping, TensorBoard
      from tensorflow.keras.metrics import Precision, Recall #for Precision and Recall metric

      import tensorflow
      tensorflow.random.set_seed(47)
      np.random.seed(47)
      

      # Define precision as a metric object
      precision_metric = Precision()
      recall_metric = Recall()

      CNN_model = Sequential() 
      # Firstly, we will add an embedding layer which will convert each word into vector 
      # then set the hyperparameters of the layer
      # We use Conv1D because sentences have one dimension: Convolutional layer is 8x300 and filter(kernel_size)=32 3x3
      CNN_model.add(Conv1D(32, kernel_size=16, activation='elu', padding='same', input_shape=(max_vocab_token, embedding_dim)))

      CNN_model.add(Conv1D(48, kernel_size=12, activation='elu', padding='same'))
      CNN_model.add(Conv1D(64, kernel_size=8, activation='relu', padding='same'))
      CNN_model.add(MaxPooling1D(pool_size=4)) # Down sampling

      # Add LSTM layer whit 512 neuron & Dropout--> use for prevent of overfitting
      CNN_model.add(Bidirectional(LSTM(512, dropout=0.1, recurrent_dropout=0.2)))

      # "Dense" refers to a fully connected layer
      CNN_model.add(Dense(512, activation='sigmoid')) # sigmoid --> use for binary classification
      CNN_model.add(Dropout(0.5)) # Dropout--> use for prevent of overfitting
      CNN_model.add(Dense(512, activation='sigmoid'))
      CNN_model.add(Dropout(0.6))
      CNN_model.add(Dense(512, activation='sigmoid'))
      CNN_model.add(Dropout(0.7))

      # Dense 2 --> this layer is used to Decision between two classes.
      CNN_model.add(Dense(2, activation='softmax')) # softmax --> Returns the probability of a comment for each class.

      # categorical_crossentropy cost function is used for multi-category classification problems.
      # Adam's optimization algorithm parameters is used and lr=0.0001 determine the learning rate and decay=1e-6 determine step size reduction rate
      CNN_model.compile(loss='categorical_crossentropy', optimizer=Adam(lr=0.001, decay=1e-6), metrics=['accuracy', 'AUC', precision_metric, recall_metric])
      # Show Dashboard
      #tensorboard = TensorBoard(log_dir='logs/', histogram_freq=0, write_graph=True, write_images=True)
      print(CNN_model.summary())

      #------------------------------------------------
      #@title Set batchSize and epochs and start learning

      # batch_size: is the number of data to be selected in each step
      batch_size = 500 #@param {type:"integer"}
      no_epochs = i #@param {type:"integer"}

      #start learning
      CNN_model.fit(x_train, y_train, batch_size=batch_size, shuffle=True, epochs=no_epochs,validation_data=(x_test, y_test))

      #----------------------------------
      print("*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*")
      CNN_model.metrics_names
      # model evaluate
      #CNN_model.evaluate(x=x_test, y=y_test, batch_size=32, verbose=1)

      #-------------------------------
      file.write(f"{i}," + str(CNN_model.evaluate(x=x_test, y=y_test, batch_size=32, verbose=2))+"\n")
      winsound.Beep(1000, 500)  # alarm
      time.sleep(0.5)

****************************************************************************************
Posetive count 2792
*Negetive count 304
Natural  count 569
Total dataset count 3761
Total shuffle count 1304 

Random Posetive Query:  
 ['تعویض سیم کارت همراه اول غیر حضوری', 1.0]
Random Negetive Query:  
 ['جزئیات قبض همراه اول', 2.0]
Random unknown  Query:  
 ['بامیلو', 3.0]
data_item is: 1303 

(1043, 8, 300) (260, 8, 300) (1043, 2) (260, 2)
Model: "sequential_141"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv1d_423 (Conv1D)         (None, 8, 32)             153632    
                                                                 
 conv1d_424 (Conv1D)         (None, 8, 48)             18480     
                                                                 
 conv1d_425 (Conv1D)         (None, 8, 64)             24640     
                                                                 
 max_poolin

d:\IT PHD 1403\Sentiment-Analyzer\Persian-Opinion-Mining-and-Sentiment-Analysis\myenv\lib\site-packages\keras\optimizers\legacy\adam.py:117: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super().__init__(name, **kwargs)


3/3 [==============================] - 8s 733ms/step - loss: 0.7772 - accuracy: 0.6213 - auc: 0.6642 - precision_141: 0.6213 - recall_141: 0.6213 - val_loss: 0.5937 - val_accuracy: 0.7423 - val_auc: 0.7423 - val_precision_141: 0.7423 - val_recall_141: 0.7423
Epoch 2/5
3/3 [==============================] - 1s 266ms/step - loss: 0.7187 - accuracy: 0.7057 - auc: 0.7435 - precision_141: 0.7057 - recall_141: 0.7057 - val_loss: 0.6496 - val_accuracy: 0.7423 - val_auc: 0.7423 - val_precision_141: 0.7423 - val_recall_141: 0.7423
Epoch 3/5
3/3 [==============================] - 1s 332ms/step - loss: 0.7601 - accuracy: 0.7066 - auc: 0.7465 - precision_141: 0.7066 - recall_141: 0.7066 - val_loss: 0.6650 - val_accuracy: 0.7423 - val_auc: 0.7423 - val_precision_141: 0.7423 - val_recall_141: 0.7423
Epoch 4/5
3/3 [==============================] - 1s 287ms/step - loss: 0.7365 - accuracy: 0.7162 - auc: 0.7555 - precision_141: 0.7162 - recall_141: 0.7162 - val_loss: 0.6579 - val_accuracy: 0.7423 - val

d:\IT PHD 1403\Sentiment-Analyzer\Persian-Opinion-Mining-and-Sentiment-Analysis\myenv\lib\site-packages\keras\optimizers\legacy\adam.py:117: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super().__init__(name, **kwargs)


3/3 [==============================] - 9s 1s/step - loss: 1.0838 - accuracy: 0.4650 - auc: 0.4414 - precision_142: 0.4650 - recall_142: 0.4650 - val_loss: 0.5893 - val_accuracy: 0.7577 - val_auc: 0.7582 - val_precision_142: 0.7577 - val_recall_142: 0.7577
Epoch 2/10
3/3 [==============================] - 1s 314ms/step - loss: 0.8987 - accuracy: 0.5666 - auc: 0.5799 - precision_142: 0.5666 - recall_142: 0.5666 - val_loss: 0.5604 - val_accuracy: 0.7577 - val_auc: 0.7577 - val_precision_142: 0.7577 - val_recall_142: 0.7577
Epoch 3/10
3/3 [==============================] - 1s 344ms/step - loss: 0.7591 - accuracy: 0.6568 - auc: 0.7082 - precision_142: 0.6568 - recall_142: 0.6568 - val_loss: 0.6216 - val_accuracy: 0.7577 - val_auc: 0.7577 - val_precision_142: 0.7577 - val_recall_142: 0.7577
Epoch 4/10
3/3 [==============================] - 1s 342ms/step - loss: 0.7301 - accuracy: 0.7200 - auc: 0.7547 - precision_142: 0.7200 - recall_142: 0.7200 - val_loss: 0.6839 - val_accuracy: 0.7577 - val

...

d:\IT PHD 1403\Sentiment-Analyzer\Persian-Opinion-Mining-and-Sentiment-Analysis\myenv\lib\site-packages\keras\optimizers\legacy\adam.py:117: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super().__init__(name, **kwargs)


3/3 [==============================] - 8s 814ms/step - loss: 1.1316 - accuracy: 0.4353 - auc: 0.4092 - precision_180: 0.4353 - recall_180: 0.4353 - val_loss: 0.6469 - val_accuracy: 0.7923 - val_auc: 0.7923 - val_precision_180: 0.7923 - val_recall_180: 0.7923
Epoch 2/200
3/3 [==============================] - 1s 355ms/step - loss: 0.8611 - accuracy: 0.5340 - auc: 0.5619 - precision_180: 0.5340 - recall_180: 0.5340 - val_loss: 0.5235 - val_accuracy: 0.7923 - val_auc: 0.7923 - val_precision_180: 0.7923 - val_recall_180: 0.7923
Epoch 3/200
3/3 [==============================] - 1s 344ms/step - loss: 0.7596 - accuracy: 0.6347 - auc: 0.6666 - precision_180: 0.6347 - recall_180: 0.6347 - val_loss: 0.5175 - val_accuracy: 0.7923 - val_auc: 0.7923 - val_precision_180: 0.7923 - val_recall_180: 0.7923
Epoch 4/200
3/3 [==============================] - 1s 324ms/step - loss: 0.7133 - accuracy: 0.6932 - auc: 0.7373 - precision_180: 0.6932 - recall_180: 0.6932 - val_loss: 0.5557 - val_accuracy: 0.7923

d:\IT PHD 1403\Sentiment-Analyzer\Persian-Opinion-Mining-and-Sentiment-Analysis\myenv\lib\site-packages\keras\optimizers\legacy\adam.py:117: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super().__init__(name, **kwargs)


3/3 [==============================] - 8s 761ms/step - loss: 1.3648 - accuracy: 0.3615 - auc: 0.3227 - precision_181: 0.3615 - recall_181: 0.3615 - val_loss: 0.8802 - val_accuracy: 0.2000 - val_auc: 0.2000 - val_precision_181: 0.2000 - val_recall_181: 0.2000
Epoch 2/205
3/3 [==============================] - 1s 294ms/step - loss: 1.0946 - accuracy: 0.4295 - auc: 0.4124 - precision_181: 0.4295 - recall_181: 0.4295 - val_loss: 0.6098 - val_accuracy: 0.8000 - val_auc: 0.8000 - val_precision_181: 0.8000 - val_recall_181: 0.8000
Epoch 3/205
3/3 [==============================] - 1s 285ms/step - loss: 0.9201 - accuracy: 0.5139 - auc: 0.5207 - precision_181: 0.5139 - recall_181: 0.5139 - val_loss: 0.5089 - val_accuracy: 0.8000 - val_auc: 0.8000 - val_precision_181: 0.8000 - val_recall_181: 0.8000
Epoch 4/205
3/3 [==============================] - 1s 376ms/step - loss: 0.7690 - accuracy: 0.6299 - auc: 0.6634 - precision_181: 0.6299 - recall_181: 0.6299 - val_loss: 0.5060 - val_accuracy: 0.8000

KeyboardInterrupt: 

## Step 3) Config & Compile & Fit the LSTM Model

Now we will create our LSTM model then feed it our Train data

This code will help you build a neural network model with LSTM, which is capable of predicting the level of delusion, i.e. the dangerousness of an opinion.
First, we create the LSTM_model and add layers to it sequentially.
First, a Conv1D layer is added to the model, which is used to convert each word into a suitable vector.
In this model, two more Conv1D layers have been added to the model, which use 3x3 size filters.
A MaxPooling1D layer with a window size of 3 is also added to the model because it helps reduce dimensionality (i.e. ease of processing).
Then an LSTM layer with 512 neurons is added to the model, which uses long sentences for prediction.
Then three perceptron layers with sigmoid activations are added to the model. The dimensions of these layers are 512, 512 and 512 respectively.
To prevent overfitting, three Dropout layers with coefficients of 0.2 and 0.25 are used.
Finally, a Dense layer is added to the model which is the number of desired decision output (in this case 2) and finally softmax is used as activation which returns the probabilities of the classes.
The compile function is used to set the parameters of the model, where categorical_crossentropy is used as a loss function and is used for Adam optimization.
At the end, by using model print, we get a summary of the model structure.

In [ ]:
import time
import winsound

# باز کردن فایل با بافرینگ خطی (هر خط بلافاصله نوشته شود)
with open("output.txt", "w", buffering=1) as file:
    for i in range(20, 301, 5):
        file.write(f"{i}\n")
        winsound.Beep(1000, 500)  # پخش آلارم
        time.sleep(0.5)

KeyboardInterrupt: 

## Step 4) Evaluate and Save the Model

in this step we evaluate LSTM model loss and accuracy metric
loss: 0.5849 - accuracy: 0.8333

In [ ]:
#CNN_model.metrics_names

['loss', 'accuracy', 'auc', 'precision_4', 'recall_4']

### ارزیابی کردن  مدل جدید با دیتاست تست قدیمی بدون آنکه لکسیون به دیتاست اضافه شده باشد در این محل

In [8]:
CNN_model.metrics_names
# model evaluate
CNN_model.evaluate(x=x_test, y=y_test, batch_size=32, verbose=1)

9/9 [==============================] - 0s 19ms/step - loss: 0.1936 - accuracy: 0.9477 - auc: 0.9604 - precision: 0.9477 - recall: 0.9477


[0.19357769191265106,
 0.9477351903915405,
 0.9603977203369141,
 0.9477351903915405,
 0.9477351903915405]

In [17]:
# Save, zip and download the model for future use
CNN_model.save('learned-query-sentiment-fasttext.model') 

!zip -r learned-query-sentiment-fasttext.model.zip learned-query-sentiment-fasttext.model

from google.colab import files
files.download('learned-query-sentiment-fasttext.model.zip')

INFO:tensorflow:Assets written to: learned-query-sentiment-fasttext.model\assets


INFO:tensorflow:Assets written to: learned-query-sentiment-fasttext.model\assets
'zip' is not recognized as an internal or external command,
operable program or batch file.


ModuleNotFoundError: No module named 'google.colab'

## Step 5) Test the Model

### There are three ways to test the model

In [18]:
#@title using model 1

user_text = "\u062E\u06CC\u0644\u06CC \u06AF\u0648\u0634\u06CC\u0647 \u062E\u0648\u0628\u06CC\u0647. \u062A\u0634\u062E\u06CC\u0635 \u0686\u0647\u0631\u0647 \u062F\u0627\u0631\u0647. \u062F\u0627\u062E\u0644 \u062C\u0639\u0628\u0647 \u06A9\u0627\u0648\u0631 \u06AF\u0648\u0634\u06CC \u0648 \u0645\u062D\u0627\u0641\u0638 \u0635\u0641\u062D\u0647 \u062F\u0627\u0631\u0647. \u0645\u0646 \u062F\u06CC\u0631\u0648\u0632 \u0628\u0647 \u062F\u0633\u062A\u0645 \u0631\u0633\u06CC\u062F\u0647 \u0639\u0627\u0644\u06CC\u0647 \u0645\u0631\u0633\u06CC \u0627\u0632 \u062F\u06CC\u062C\u06CC \u06A9\u0627\u0644\u0627" #@param {type:"string"}
from IPython.core.display import display, HTML
_normalizer = hazm.Normalizer()
if not user_text=="":
  normal_text = _normalizer.normalize(user_text)
  tokenized_text = hazm.word_tokenize(normal_text)
  
  # create and Prepare three dimension tensor (1,20,300) with zero value : (1,number_of_words, dimension_of_fasttext)
  vector_text = np.zeros((1,max_vocab_token,embedding_dim),dtype=K.floatx())


  for vocabs in range(0,len(tokenized_text)):
    if vocabs >= max_vocab_token:
      break # If the comment is more than twenty words, only the first twenty words will be considered
    if tokenized_text[vocabs] not in fasttext_model.words:
      continue # If vocab does not exist in fasttext, every 300 elements of that word's vector remain zero
    
    vector_text[0, vocabs, :] = fasttext_model.get_word_vector(tokenized_text[vocabs])

  # print(vector_text.shape)
  # print(vector_text)
  result = CNN_model.predict(vector_text) # the result has two element: [0][1] and [0][0]
  pos_percent = str(int(result[0][1]*100))+" % 😍"
  neg_percent = str(int(result[0][0]*100))+" % 🤕"
  display(HTML("<div style='text-align: center'><div style='display:inline-block'><img height='64px' width='64px' src='https://images.rawpixel.com/image_png_1000/cHJpdmF0ZS9sci9pbWFnZXMvd2Vic2l0ZS8yMDIyLTEwL3JtNTg2LWlubG92ZWZhY2UtMDFfMS1sOWQzYzlxMC5wbmc.png'/><h4>{}</h4></div> | <div style='display:inline-block'><img height='64px' width='64px' src='https://images.rawpixel.com/image_png_1000/cHJpdmF0ZS9sci9pbWFnZXMvd2Vic2l0ZS8yMDIyLTEwL3JtNTg2LWNyeWluZ2ZhY2UtMDFfMi1sOWQzYnh0MC5wbmc.png'/><h4>{}</h4></div></div>".format(pos_percent,neg_percent)))
else:
  print("Please enter your text")

C:\Users\Administrator\AppData\Local\Temp\ipykernel_14616\2897243695.py:4: DeprecationWarning: Importing display from IPython.core.display is deprecated since IPython 7.14, please import from IPython.display
  from IPython.core.display import display, HTML


1/1 [==============================] - 1s 558ms/step


In [19]:
#@title using model 2

user_text = "\u062E\u06CC\u0644\u06CC \u06AF\u0648\u0634\u06CC\u0647 \u062E\u0648\u0628\u06CC\u0647. \u062A\u0634\u062E\u06CC\u0635 \u0686\u0647\u0631\u0647 \u062F\u0627\u0631\u0647. \u062F\u0627\u062E\u0644 \u062C\u0639\u0628\u0647 \u06A9\u0627\u0648\u0631 \u06AF\u0648\u0634\u06CC \u0648 \u0645\u062D\u0627\u0641\u0638 \u0635\u0641\u062D\u0647 \u062F\u0627\u0631\u0647. \u0645\u0646 \u062F\u06CC\u0631\u0648\u0632 \u0628\u0647 \u062F\u0633\u062A\u0645 \u0631\u0633\u06CC\u062F\u0647 \u0639\u0627\u0644\u06CC\u0647 \u0645\u0631\u0633\u06CC \u0627\u0632 \u062F\u06CC\u062C\u06CC \u06A9\u0627\u0644\u0627" #@param {type:"string"}
from IPython.core.display import display, HTML
_normalizer = hazm.Normalizer()
if not user_text=="":
  normal_text = _normalizer.normalize(user_text)
  tokenized_text = hazm.word_tokenize(normal_text)
  
  # create and Prepare three dimension tensor (1,20,300) with zero value : (1,number_of_words, dimension_of_fasttext)
  vector_text = np.zeros((1,max_vocab_token,embedding_dim),dtype=K.floatx())

  for vocabs in range(0,len(tokenized_text)):
    if vocabs >= max_vocab_token:
      break # If the comment is more than twenty words, only the first twenty words will be considered
    if tokenized_text[vocabs] not in fasttext_model.words:
      continue # If vocab does not exist in fasttext, every 300 elements of that word's vector remain zero
    
    vector_text[0, vocabs, :] = fasttext_model.get_word_vector(tokenized_text[vocabs])

  # print(vector_text.shape)
  # print(vector_text)
  result = CNN_model.predict(vector_text) # the result has two element: [0][1] and [0][0]
  pos_percent = str(int(result[0][1]*100))+" % 😍"
  neg_percent = str(int(result[0][0]*100))+" % 🤕"
  display(HTML("<div style='text-align: center'><div style='display:inline-block'><img height='64px' width='64px' src='https://image.flaticon.com/icons/svg/260/260205.svg'/><h4>{}</h4></div> | <div style='display:inline-block'><img height='64px' width='64px' src='https://image.flaticon.com/icons/svg/260/260206.svg'/><h4>{}</h4></div></div>".format(pos_percent,neg_percent)))
else:
  print("Please enter your text")

1/1 [==============================] - 0s 29ms/step


C:\Users\Administrator\AppData\Local\Temp\ipykernel_14616\1865725138.py:4: DeprecationWarning: Importing display from IPython.core.display is deprecated since IPython 7.14, please import from IPython.display
  from IPython.core.display import display, HTML


In [20]:
#@title using model 3

user_text = "\u062E\u06CC\u0644\u06CC \u062C\u0627\u0644\u0628\u0647 \u0627\u06CC\u0646 \u0645\u0648\u0628\u0627\u06CC\u0644 \u0627\u0635\u0644\u0627 \u0647\u0645\u0647 \u0686\u06CC \u062A\u0645\u0627\u0645\u0647 \u0645\u0646 \u06A9\u0647 \u067E\u0633\u0646\u062F\u06CC\u062F\u0645 \u0627\u06CC\u0646 \u0645\u0648\u0628\u0627\u06CC\u0644 \u0632\u06CC\u0628\u0627 \u0631\u0648" #@param {type:"string"}
from IPython.core.display import display, HTML
_normalizer = hazm.Normalizer()
if not user_text=="":
  normal_text = _normalizer.normalize(user_text)
  tokenized_text = hazm.word_tokenize(normal_text)
  vector_text = np.zeros((1,max_vocab_token,embedding_dim),dtype=K.floatx())
  for vocabs in range(0,len(tokenized_text)):
    if vocabs >= max_vocab_token:
      break
    if tokenized_text[vocabs] not in fasttext_model.words:
      continue
    
    vector_text[0, vocabs, :] = fasttext_model.get_word_vector(tokenized_text[vocabs])
  # print(x_text_for_test_words.shape)
  # print(text_for_test_words)
  result = CNN_model.predict(vector_text)
  pos_percent = str(int(result[0][1]*100))+" % "
  neg_percent = str(int(result[0][0]*100))+" % "
  display(HTML("<div style='text-align: center'><div style='display:inline-block'><img height='64px' width='64px' src='https://image.flaticon.com/icons/svg/260/260205.svg'/><h4>{}</h4></div> | <div style='display:inline-block'><img height='64px' width='64px' src='https://image.flaticon.com/icons/svg/260/260206.svg'/><h4>{}</h4></div></div>".format(pos_percent,neg_percent)))
else:
  print("Please enter your text")

1/1 [==============================] - 0s 28ms/step


C:\Users\Administrator\AppData\Local\Temp\ipykernel_14616\1199773519.py:4: DeprecationWarning: Importing display from IPython.core.display is deprecated since IPython 7.14, please import from IPython.display
  from IPython.core.display import display, HTML
